In [3]:
%pip install smolagents pandas langchain langchain-community sentence-transformers faiss-cpu datasets rank_bm25 --upgrade -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

/home/bredda/workspace/genai-talk-notebooks/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filter: 100%|██████████| 2647/2647 [00:00<00:00, 49027.05 examples/s]


In [4]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=10
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

retriever_tool = RetrieverTool(docs_processed)

In [6]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(
    tools=[retriever_tool], model=HfApiModel(), max_iterations=4, verbose=True
)
agent_output = agent.run("For a transformers model training, which is slower, the forward or the backward pass?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ For a transformers model training, which is slower, the forward or the backward pass?                           │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To answer this question, I need to retrieve information about the forward and backward passes in the      
context of training a transformers model. I'll use the `retriever` tool to get relevant information from the       
transformers documentation.                                                                                        
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
forward_pass_info = retriever(query="forward pass in transformers model")                                          
backward_pass_info = retriever(query="backward pass in transformers model")                                        
print("Forward pass info:", forward_pass_info)                                                                     
print("Backward pass info:", backward_pass_info)                                                                   
```<end_code>                                                                                                      

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 forward_pass_info = retriever(query="forward pass in transformers model")                                   │
│   2 backward_pass_info = retriever(query="backward pass in transformers model")                                 │
│   3 print("Forward pass info:", forward_pass_info)                                                              │
│   4 print("Backward pass info:", backward_pass_info)                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Execution logs:
Forward pass info: 
Retrieved documents:


===== Document 0 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers

===== Document 1 =====
4.  [ ] Created script that successfully runs forward pass using
    original repository and checkpoint

5.  [ ] Successfully opened a PR and added the model skeleton to Transformers

6.  [ ] Successfully converted original checkpoint to Transformers
    checkpoint

7.  [ ] Successfully ran forward pass in Transformers that gives
    identical output to original checkpoint

8.  [ ] Finished model tests in Transformers

9.  [ ] Successfully added Tokenizer in Transformers

===== Document 2 =====
model_inputs = tokenizer(src_text, text_target=tgt_text, return_tensors="pt")

model(**model_inputs)  # forward pass
```

- Generation

===== Document 3 =====
>>> model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-en-ro")
>>> # forward pass
>>> model(**inputs)
```

- Generation

  While generating the target text set the `decoder_start_token_id` to the target language id. The following
  example shows how to translate English to Romanian using the *facebook/mbart-large-en-ro* model.

```python
>>> from transformers import MBartForConditionalGeneration, MBartTokenizer

===== Document 4 =====
```python
from transformers import Tool


class HFModelDownloadsTool(Tool):
    pass
```

===== Document 5 =====
Next, regarding the debugging strategy, there are generally a few from
which to choose from:

-   Decompose the original model into many small testable components and
    run a forward pass on each of those for verification
-   Decompose the original model only into the original *tokenizer* and
    the original *model*, run a forward pass on those, and use
    intermediate print statements or breakpoints for verification

===== Document 6 =====
Next, regarding the debugging strategy, there are generally a few from which to choose from:

- Decompose the original model into many small testable components and run a forward pass on each of those for
  verification
- Decompose the original model only into the original *tokenizer* and the original *model*, run a forward pass on
  those, and use intermediate print statements or breakpoints for verification

===== Document 7 =====
layer is falsely activated during the forward pass, *i.e.* pass *self.training* to [PyTorch's functional 
dropout](https://pytorch.org/docs/stable/nn.functional.html?highlight=dropout#torch.nn.functional.dropout)

===== Document 8 =====
- For languages with a Roman alphabet, such as English or French, the tokenizer can be used directly to 
pre-process the text inputs. The following code example runs a forward pass using the MMS-TTS English checkpoint:

```python
import torch
from transformers import VitsTokenizer, VitsModel, set_seed

tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")
model = VitsModel.from_pretrained("facebook/mms-tts-eng")

===== Document 9 =====
Having managed to correctly load the pretrained weights into the 🤗 Transformers implementation, you should now 
make
sure that the forward pass is correctly implemented. In [Get familiar with the original 
repository](#34-run-a-pretrained-checkpoint-using-the-original-repository), you have already created a script that 
runs a forward
pass of the model using the original repository. Now you should write an analogous script using the 🤗 Transformers
Backward pass info: 
Retrieved documents:


===== Document 0 =====
Saving all activations from the forward pass in order to compute the gradients during th

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the retrieved information, the backward pass generally introduces a considerable computational   
overhead compared to the forward pass, especially due to the need to recalculate intermediate activations and the  
increased number of floating-point operations (flops) for certain layers like convolutions and linear layers.      
                                                                                                                   
Therefore, it is reasonable to conclude that the backward pass is slower than the forward pass for a transformers  
model training. I'll provide the final answer using the `final_answer` tool.                                       
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("backward pass")                                                                                      
```<end_code>                                                                                                      

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 final_answer("backward pass")                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: backward pass

[Step 1: Duration 3.36 seconds| Input tokens: 5,795 | Output tokens: 215]

Final output:
backward pass
